# project log

## 20-Jan-2026: 
### Explored the https://clinicaltrials.gov/data-api/api webstie to investigate the possible functions we could write

In [5]:
library(httr)
library(jsonlite)

# 1. Build the URL
url <- "https://clinicaltrials.gov/api/v2/studies?query.term=diabetes&pageSize=5"

# 2. Download the data
response <- GET(url)

# 3. Convert JSON to R list
data <- fromJSON(content(response, "text"), simplifyVector = FALSE)

# 4. Look at what we got
names(data)
studies <- data$studies

# Look at the first study
studies[[1]]


No encoding supplied: defaulting to UTF-8.



[1] "studies"       "nextPageToken"

$protocolSection
$protocolSection$identificationModule
$protocolSection$identificationModule$nctId
[1] "NCT04441658"

$protocolSection$identificationModule$orgStudyIdInfo
$protocolSection$identificationModule$orgStudyIdInfo$id
[1] "DFSC-2020(CR)-02"


$protocolSection$identificationModule$organization
$protocolSection$identificationModule$organization$fullName
[1] "Shanghai East Hospital"

$protocolSection$identificationModule$organization$class
[1] "OTHER"


$protocolSection$identificationModule$briefTitle
[1] "Efficacy and Safety of Umbilical Cord Mesenchymal Stem Cells Transplantation in Patients With Type 2 Diabetes Mellitus"

$protocolSection$identificationModule$officialTitle
[1] "Efficacy and Safety of Umbilical Cord Mesenchymal Stem Cells Transplantation in Patients With Type 2 Diabetes Mellitus"


$protocolSection$statusModule
$protocolSection$statusModule$statusVerifiedDate
[1] "2020-06"

$protocolSection$statusModule$overallStatus
[1] "UNKNOWN"

$protocolSection$statusModule$lastKnownStatus
[1] "RECRUITING"

$protocolSection$statusModule$expandedAccessInfo
$protocolSection$statusModule$expandedAccessInfo$hasExpandedAccess
[1] FALSE


$protocolSection$statusModule$startDateStruct
$protocolSection$statusModule$startDateStruct$date
[1] "2020-04-10"

$protocolSection$statusModule$startDateStruct$type
[1] "ACTUAL"


$protocolSection$statusModule$primaryCompletionDateStruct
$protocolSection$statusModule$primaryCompletionDateStruct$date
[1] "2022-07"

$protocolSection$statusModule$primaryCompletionDateStruct$type
[1] "ESTIMATED"


$protocolSection$statusModule$completionDateStruct
$protocolSection$statusModule$completionDateStruct$date
[1] "2022-12"

$protocolSection$statusModule$completionDateStruct$type
[1] "ESTIMATED"


$protocolSection$statusModule$studyFirstSubmitDate
[1] "2020-06-19"

$protocolSection$statusModule$studyFirstSubmitQcDate
[1] "2020-06-19"

$protocolSection$statusModule$studyFirstPostDateStruct
$protocolSection$statusModule$studyFirstPostDateStruct$date
[1] "2020-06-22"

$protocolSection$statusModule$studyFirstPostDateStruct$type
[1] "ACTUAL"


$protocolSection$statusModule$lastUpdateSubmitDate
[1] "2020-06-19"

$protocolSection$statusModule$lastUpdatePostDateStruct
$protocolSection$statusModule$lastUpdatePostDateStruct$date
[1] "2020-06-22"

$protocolSection$statusModule$lastUpdatePostDateStruct$type
[1] "ACTUAL"



$protocolSection$sponsorCollaboratorsModule
$protocolSection$sponsorCollaboratorsModule$responsibleParty
$protocolSection$sponsorCollaboratorsModule$responsibleParty$type
[1] "SPONSOR"


$protocolSection$sponsorCollaboratorsModule$leadSponsor
$protocolSection$sponsorCollaboratorsModule$leadSponsor$name
[1] "Shanghai East Hospital"

$protocolSection$sponsorCollaboratorsModule$leadSponsor$class
[1] "OTHER"



$protocolSection$oversightModule
$protocolSection$oversightModule$isFdaRegulatedDrug
[1] FALSE

$protocolSection$oversightModule$isFdaRegulatedDevice
[1] FALSE


$protocolSection$descriptionModule
$protocolSection$descriptionModule$briefSummary
[1] "This study is a 24-week single-center, randomized, double-blind, placebo-controlled trial. The trial includes a 3-week early screening and lifestyle education period, 6-week treatment period, and 18-week follow-up period. Chinese type 2 diabetic subjects receiving traditional hypoglycemic treatment were randomly assigned to umbilical cord mesenchymal stem cell or placebo infusion therapy to observe the efficacy and safety of umbilical cord mesenchymal stem cell infusion therapy."

$protocolSection$descriptionModule$detailedDescription
[1] "This study is a 24-week single-center, randomized, double-blind, placebo-controlled trial. The trial includes a 3-week early screening and lifestyle education period, 6-week treatment period, and 18-week follow-up period. Chinese type 2 diabetic subjects receiving traditional hypoglycemic treatment（including insulin） were randomly assigned to umbilical cord mesenchymal stem cell or placebo infusion therapy 

In [6]:

for (i in seq_along(studies)) {
  study <- studies[[i]]

  nct_id <- study$protocolSection$identificationModule$nctId
  title  <- study$protocolSection$identificationModule$briefTitle
  status <- study$protocolSection$statusModule$overallStatus

  cat("\n------------------------\n")
  cat("NCT ID:", nct_id, "\n")
  cat("Title:", title, "\n")
  cat("Status:", status, "\n")
}



------------------------
NCT ID: NCT04441658 
Title: Efficacy and Safety of Umbilical Cord Mesenchymal Stem Cells Transplantation in Patients With Type 2 Diabetes Mellitus 
Status: UNKNOWN 

------------------------
NCT ID: NCT03162926 
Title: A Safety and Tolerability Study of VC-02™ Combination Product in Subjects With Type 1 Diabetes Mellitus 
Status: COMPLETED 

------------------------
NCT ID: NCT03797885 
Title: A Study of People With Type 2 Diabetes Who Have Diseases That Affect the Heart and Blood Vessels. The Study Will Also Look at the Treatment of a Group of These People in Hospitals in Portugal 
Status: COMPLETED 

------------------------
NCT ID: NCT01373476 
Title: Multicentre, Randomized, Controlled Trial of Qideng Mingmu Capsule in The Treatment of Diabetic Retinopathy 
Status: COMPLETED 

------------------------
NCT ID: NCT01686568 
Title: Omega-3 Fatty Acids and Insulin Sensitivity 
Status: COMPLETED 


- Noticed here the Json is organized like a nested list. There are modules that group the similar fields together.
- what are the possible modules

## 21-Jan-2026

- Collaborated with team members and threw around some ideas.
- what fields should we add?
- what modules do we add?

In [1]:
library(httr)
library(jsonlite)
library(dplyr)
library(purrr)
library(tibble)

`%||%` <- function(x, y) if (is.null(x)) y else x

ct_search_studies <- function(query = NULL,
                              condition = NULL,
                              intervention = NULL,
                              page_size = 100,
                              page_token = NULL) {

  base_url <- "https://clinicaltrials.gov/api/v2/studies"

  # Build query parameters
  params <- list(
    pageSize = page_size,
    format = "json"
  )

  if (!is.null(query)) {
    params[["query.term"]] <- query
  }

  if (!is.null(condition)) {
    params[["query.cond"]] <- condition
  }

  if (!is.null(intervention)) {
    params[["query.intr"]] <- intervention
  }

  if (!is.null(page_token)) {
    params[["pageToken"]] <- page_token
  }

  # Send request
  resp <- GET(base_url, query = params)
  stop_for_status(resp)

  txt <- content(resp, as = "text", encoding = "UTF-8")
  json <- fromJSON(txt, simplifyVector = FALSE)

  studies <- json$studies %||% list()

  # Extract a few useful fields into a tidy table
  out <- tibble(
    nct_id = map_chr(studies, ~ .x$protocolSection$identificationModule$nctId %||% NA_character_),
    title  = map_chr(studies, ~ .x$protocolSection$identificationModule$briefTitle %||% NA_character_),
    status = map_chr(studies, ~ .x$protocolSection$statusModule$overallStatus %||% NA_character_),
    phase  = map_chr(studies, ~ paste(.x$protocolSection$designModule$phaseList$phase, collapse = ", ") %||% NA_character_),
    start_date = map_chr(studies, ~ .x$protocolSection$statusModule$startDateStruct$date %||% NA_character_)
  )

  list(
    data = out,
    next_page_token = json$nextPageToken %||% NULL,
    total_count = json$totalCount %||% NA_integer_
  )
}


Warning message:
"package 'httr' was built under R version 4.5.2"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'purrr'


The following object is masked from 'package:jsonlite':

    flatten




In [2]:
res <- ct_search_studies(condition = "diabetes", page_size = 20)

res$data

nct_id,title,status,phase,start_date
<chr>,<chr>,<chr>,<chr>,<chr>
NCT02951260,"The Effects of Metformin on Self-selected Exercise Intensity, Physical Fitness and Exercise-induced AMPK-activation",COMPLETED,,2016-10
NCT00286624,"Anti-Thymocyte Globulin, Cyclosporine, and RAD in Islet Transplantation",COMPLETED,,2003-03
NCT00859079,Efficacy of GLP-1 Infusion in Comparison to an Insulin Infusion Protocol to Reach Normoglycemia Type 2 Diabetic Patients,COMPLETED,,2006-06
NCT00480779,Evaluation of Group Lifestyle Balance DVD in Primary Care Practice,COMPLETED,,2007-06
NCT03885362,Assessment of the Accuracy of Continuous Glucose Sensors in People With Diabetes Undergoing Haemodialysis,COMPLETED,,2019-12-11
NCT04089462,Effects of Frequency and Duration of Exercise in People With Type 1 Diabetes A Randomized Crossover Study,COMPLETED,,2019-09-30
NCT03810846,Feasibility and Safety of Walking Football in Patients With Type 2 Diabetes,COMPLETED,,2018-09-17
NCT00136188,Nitric Oxide (NO) Activity and Diabetic Nephropathy,COMPLETED,,2009-08
NCT04919603,Impact of Exercise Intervention on the Phenome,UNKNOWN,,2021-06-15


## 28-Jan-2026

- Realized filtering dates is very tedious the dates have to be in a specific format or else it will break. 
- Tested out teammates code and cleaned up the output tables of his API .
- I am experimenting with nested tables but i think its too complex and not very user friendly

In [2]:
suppressWarnings(devtools::load_all())
spec <- list(
  query = "cancer",
  max_records = 10,
  phase = "Phase 2",
  country = "Canada",
  from_date = "2022-01-01",
  to_date = "2023-01-01",
  date_filter = "StartDate"
)
result <- trials_run_spec(spec)
# str(result)

ℹ Loading tidytrials

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [16]:
`%||%` <- function(a, b) if (is.null(a) || length(a) == 0) b else a

get_in <- function(x, path, default = NA_character_) {
  for (nm in path) {
    if (is.null(x[[nm]])) return(default)
    x <- x[[nm]]
  }
  if (is.null(x) || length(x) == 0) return(default)
  if (is.atomic(x)) return(as.character(x)[1])
  default
}


In [19]:
library(tibble)
library(purrr)
library(dplyr)

studies_to_table <- function(studies) {
  map_dfr(studies, function(st) {

    # pull lists safely
    conds <- st$protocolSection$conditionsModule$conditions %||% character(0)
    locs  <- st$protocolSection$contactsLocationsModule$locations %||% list()

    # collapse conditions
    condition_all <- if (length(conds) == 0) NA_character_ else paste(unlist(conds), collapse = "; ")

    # derive countries + site count
    countries <- unique(vapply(locs, function(z) z$country %||% NA_character_, character(1)))
    countries <- countries[!is.na(countries)]
    country_all <- if (length(countries) == 0) NA_character_ else paste(countries, collapse = "; ")
    n_sites <- length(locs)

    tibble(
      nct_id = get_in(st, c("protocolSection","identificationModule","nctId")),
      org_study_id = get_in(st, c("protocolSection","identificationModule","orgStudyIdInfo","id")),
      brief_title = get_in(st, c("protocolSection","identificationModule","briefTitle")),
      official_title = get_in(st, c("protocolSection","identificationModule","officialTitle")),

      overall_status = get_in(st, c("protocolSection","statusModule","overallStatus")),
      why_stopped = get_in(st, c("protocolSection","statusModule","whyStopped")),

      start_date = get_in(st, c("protocolSection","statusModule","startDateStruct","date")),
      primary_completion_date = get_in(st, c("protocolSection","statusModule","primaryCompletionDateStruct","date")),
      completion_date = get_in(st, c("protocolSection","statusModule","completionDateStruct","date")),
      last_update_posted = get_in(st, c("protocolSection","statusModule","lastUpdatePostDateStruct","date")),

      sponsor = get_in(st, c("protocolSection","sponsorCollaboratorsModule","leadSponsor","name")),
      sponsor_class = get_in(st, c("protocolSection","sponsorCollaboratorsModule","leadSponsor","class")),

      study_type = get_in(st, c("protocolSection","designModule","studyType")),
      phase = {
        ph <- st$protocolSection$designModule$phases %||% character(0)
        if (length(ph) == 0) NA_character_ else paste(unlist(ph), collapse = ", ")
      },
      allocation = get_in(st, c("protocolSection","designModule","designInfo","allocation")),
      intervention_model = get_in(st, c("protocolSection","designModule","designInfo","interventionModel")),
      primary_purpose = get_in(st, c("protocolSection","designModule","designInfo","primaryPurpose")),
      masking = get_in(st, c("protocolSection","designModule","designInfo","maskingInfo","masking")),

      enrollment = get_in(st, c("protocolSection","designModule","enrollmentInfo","count")),
      enrollment_type = get_in(st, c("protocolSection","designModule","enrollmentInfo","type")),

      conditions = condition_all,
      countries = country_all,
      n_sites = n_sites,

      has_results = st$hasResults %||% NA
    )
  })
}

tbl <- studies_to_table(result$studies) |> distinct(nct_id, .keep_all = TRUE)
head(tbl)


nct_id,org_study_id,brief_title,official_title,overall_status,why_stopped,start_date,primary_completion_date,completion_date,last_update_posted,⋯,allocation,intervention_model,primary_purpose,masking,enrollment,enrollment_type,conditions,countries,n_sites,has_results
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<lgl>
NCT05444894,EM-301-BThal-001,EDIT-301 for Autologous Hematopoietic Stem Cell Transplant (HSCT) in Participants With Transfusion-Dependent Beta Thalassemia (TDT),"A Multicenter Study to Evaluate the Safety, Tolerability, and Efficacy of a Single Dose of Autologous Clustered Regularly Interspaced Short Palindromic Repeats Gene-edited Cluster of Differentiation 34 (CD34+) Human Hematopoietic Stem and Progenitor Cells (HSPC) (EDIT-301) in Transfusion-Dependent Beta Thalassemia (TDT)",ACTIVE_NOT_RECRUITING,NA,2022-04-29,2025-09,2025-12,2025-04-02,⋯,NA,SINGLE_GROUP,TREATMENT,NONE,9,ESTIMATED,Transfusion Dependent Beta Thalassemia; Hemoglobinopathies; Thalassemia Major; Thalassemia Intermedia,United States; Canada,8,FALSE
NCT07150091,208887 Sub Study 6,"Sub-study of Belantamab Mafodotin (GSK2857916) in Combination With Nirogacestat, Lenalidomide, and Dexamethasone in Participants With RRMM","A Phase I/II, Randomized, Open-label Platform Study Utilizing a Master Protocol to Study Belantamab Mafodotin (GSK2857916) as Monotherapy and in Combination With Anti-Cancer Treatments in Participants With Relapsed/Refractory Multiple Myeloma (RRMM)-DREAMM5 - Sub-study 6 - Belantamab Mafodotin, Nirogacestat, Lenalidomide, and Dexamethasone in Combination",ACTIVE_NOT_RECRUITING,NA,2022-07-19,2025-04-17,2027-03-11,2025-09-02,⋯,NA,SINGLE_GROUP,TREATMENT,NONE,20,ACTUAL,Multiple Myeloma,United States; Brazil; Canada; France; Germany; Greece; Mexico; South Korea,12,FALSE
NCT05421598,DRI17509,Dose Ranging Study of Amlitelimab in Adult Participants With Moderate-to-severe Asthma,"A Randomized, Double-blind, Placebo-controlled, Parallel-group, Dose Ranging Study to Assess the Efficacy, Safety, and Tolerability of Subcutaneous Amlitelimab in Adult Participants With Moderate-to-severe Asthma",COMPLETED,NA,2022-06-30,2024-10-11,2025-03-20,2025-10-09,⋯,RANDOMIZED,PARALLEL,TREATMENT,QUADRUPLE,446,ACTUAL,Asthma,United States; Argentina; Brazil; Canada; Chile; Hungary; Italy; Japan; Mexico; Poland; South Africa; South Korea; Turkey (Türkiye); United Kingdom,113,FALSE
NCT05394350,1088-002,A Study of MK-1088 as Monotherapy and in Combination With Pembrolizumab in Participants With Advanced Solid Tumors (MK-1088-002),A Phase 1/Phase 2 Study to Evaluate the Safety and Tolerability of MK-1088 as Monotherapy and in Combination With Pembrolizumab in Participants With Advanced Solid Tumors,TERMINATED,Business Reasons,2022-07-07,2023-09-07,2023-09-07,2024-11-08,⋯,NON_RANDOMIZED,PARALLEL,TREATMENT,NONE,27,ACTUAL,Advanced Solid Tumors,United States; Canada; Denmark; Israel; Switzerland,13,TRUE
NCT04765514,IIT-0012,Chemoradiotherapy Versus Biomarker-Guided Therapy for Elderly and Frail GBM Patients,A Randomized Controlled Trial of Chemo-Radiotherapy Versus Biomarker-Guided Therapy for Elderly and Frail Patients With Newly Diagnosed Glioblastoma,RECRUITING,NA,2022-07-27,2032-06,2032-06,2025-06-12,⋯,RANDOMIZED,PARALLEL,TREATMENT,NONE,121,ESTIMATED,Glioblastoma Multiforme,Canada,2,FALSE
NCT05432934,BIO-2549,Transversus Abdominis Plane Blocks With and Without Dexamethasone,Transversus Abdominis Plane Blocks With and Without Dexamethasone: A Randomized Clinical Trial in Minimally Invasive Colorectal Surgery,UNKNOWN,NA,2022-09-01,2024-03-31,2024-08-31,2023-12-07,⋯,RANDOMIZED,PARALLEL,TREATMENT,DOUBLE,60,ESTIMATED,"Colorectal Disorders; Regional Anesthesia Morbidity; Analgesia; Local Anesthetic Complication; Pain, Postoperative; Colorectal Cancer; Crohn Disease; Diverticular Diseases",Canada,1,FALSE


In [18]:
DT::datatable(tbl, options = list(pageLength = 10))



HTML widgets cannot be represented in plain text (need html)

Parameters I am keeping in the summary table:

| Column name    | Description                                          |
| -------------- | ---------------------------------------------------- |
| `nct_id`       | ClinicalTrials.gov identifier (e.g. `"NCT05444894"`) |
| `org_study_id` | Sponsor’s internal study ID                          |
| `brief_title`    | Short public title of the study |
| `official_title` | Full official study title       |
| `overall_status` | Recruitment status (e.g. `RECRUITING`, `ACTIVE_NOT_RECRUITING`) |
| `why_stopped`    | Reason study was stopped early (if applicable)                  |
| `start_date`              | Study start date                                |
| `primary_completion_date` | Primary outcome completion date                 |
| `completion_date`         | Final study completion date                     |
| `last_update_posted`      | Most recent update posted to ClinicalTrials.gov |
| `sponsor`       | Lead sponsor organization name                      |
| `sponsor_class` | Sponsor classification (`INDUSTRY`, `NIH`, `OTHER`) |
| `study_type`         | Study type (e.g. `INTERVENTIONAL`, `OBSERVATIONAL`) |
| `phase`              | Study phase(s), collapsed (e.g. `"PHASE1, PHASE2"`) |
| `allocation`         | Allocation method (`RANDOMIZED`, `NA`, etc.)        |
| `intervention_model` | Intervention model (e.g. `SINGLE_GROUP`)            |
| `primary_purpose`    | Primary purpose (e.g. `TREATMENT`)                  |
| `masking`            | Masking/blinding status                             |
| `enrollment`      | Target or actual enrollment count         |
| `enrollment_type` | Enrollment type (`ACTUAL` or `ESTIMATED`) |
| `conditions` | All study conditions collapsed into a single semicolon-separated string |
| `countries` | Unique list of countries with study sites (collapsed) |
| `n_sites`   | Total number of study locations                       |
| `has_results` | Logical indicating whether posted results are available |





### Why we are choosing these field names:

- I chose these paramters as a good starting place to start research
- these fields are important for policy creations.

created the function

In [26]:
library(tibble)
library(purrr)
library(dplyr)

#' Build a study-level table from CT.gov v2 JSON with curated module columns
#'
#' @param studies List of study records (e.g., result$studies).
#' @param modules Character vector of protocolSection module names to include.
#'   Use "all" to include all supported modules.
#'   Supported: identificationModule, statusModule, sponsorCollaboratorsModule,
#'   designModule, eligibilityModule, contactsLocationsModule, referencesModule.
#' @return Tibble with 1 row per study.
studies_to_table <- function(
  studies,
  modules = c("identificationModule", "statusModule", "sponsorCollaboratorsModule", "designModule")
) {

  extractors <- list(

    identificationModule = function(st) tibble(
      nct_id        = get_in(st, c("protocolSection","identificationModule","nctId")),
      org_study_id   = get_in(st, c("protocolSection","identificationModule","orgStudyIdInfo","id")),
      brief_title    = get_in(st, c("protocolSection","identificationModule","briefTitle")),
      official_title = get_in(st, c("protocolSection","identificationModule","officialTitle")),
      acronym        = get_in(st, c("protocolSection","identificationModule","acronym"))
    ),

    statusModule = function(st) tibble(
      overall_status            = get_in(st, c("protocolSection","statusModule","overallStatus")),
      status_verified_ym        = get_in(st, c("protocolSection","statusModule","statusVerifiedDate")),
      start_date                = get_in(st, c("protocolSection","statusModule","startDateStruct","date")),
      start_date_type           = get_in(st, c("protocolSection","statusModule","startDateStruct","type")),
      primary_completion_date   = get_in(st, c("protocolSection","statusModule","primaryCompletionDateStruct","date")),
      primary_completion_type   = get_in(st, c("protocolSection","statusModule","primaryCompletionDateStruct","type")),
      completion_date           = get_in(st, c("protocolSection","statusModule","completionDateStruct","date")),
      completion_date_type      = get_in(st, c("protocolSection","statusModule","completionDateStruct","type")),
      last_update_posted        = get_in(st, c("protocolSection","statusModule","lastUpdatePostDateStruct","date")),
      last_update_posted_type   = get_in(st, c("protocolSection","statusModule","lastUpdatePostDateStruct","type")),
      why_stopped               = get_in(st, c("protocolSection","statusModule","whyStopped"))
    ),

    sponsorCollaboratorsModule = function(st) {
      collabs <- st$protocolSection$sponsorCollaboratorsModule$collaborators %||% list()
      collab_names <- unique(vapply(collabs, function(z) z$name %||% NA_character_, character(1)))
      collab_names <- collab_names[!is.na(collab_names)]
      tibble(
        lead_sponsor_name  = get_in(st, c("protocolSection","sponsorCollaboratorsModule","leadSponsor","name")),
        lead_sponsor_class = get_in(st, c("protocolSection","sponsorCollaboratorsModule","leadSponsor","class")),
        collaborators      = if (length(collab_names) == 0) NA_character_ else paste(collab_names, collapse = "; ")
      )
    },

    designModule = function(st) {
      phases <- st$protocolSection$designModule$phases %||% character(0)
      tibble(
        study_type         = get_in(st, c("protocolSection","designModule","studyType")),
        phases             = if (length(phases) == 0) NA_character_ else paste(unlist(phases), collapse = ", "),
        allocation         = get_in(st, c("protocolSection","designModule","designInfo","allocation")),
        intervention_model = get_in(st, c("protocolSection","designModule","designInfo","interventionModel")),
        primary_purpose    = get_in(st, c("protocolSection","designModule","designInfo","primaryPurpose")),
        masking            = get_in(st, c("protocolSection","designModule","designInfo","maskingInfo","masking")),
        enrollment_count   = get_in(st, c("protocolSection","designModule","enrollmentInfo","count")),
        enrollment_type    = get_in(st, c("protocolSection","designModule","enrollmentInfo","type"))
      )
    },

    eligibilityModule = function(st) tibble(
      eligibility_criteria = get_in(st, c("protocolSection","eligibilityModule","eligibilityCriteria")),
      healthy_volunteers   = get_in(st, c("protocolSection","eligibilityModule","healthyVolunteers")),
      sex                  = get_in(st, c("protocolSection","eligibilityModule","sex")),
      minimum_age          = get_in(st, c("protocolSection","eligibilityModule","minimumAge")),
      maximum_age          = get_in(st, c("protocolSection","eligibilityModule","maximumAge")),
      std_ages             = {
        ages <- st$protocolSection$eligibilityModule$stdAges %||% character(0)
        if (length(ages) == 0) NA_character_ else paste(unlist(ages), collapse = ", ")
      }
    ),

    contactsLocationsModule = function(st) {
      locs <- st$protocolSection$contactsLocationsModule$locations %||% list()
      countries <- unique(vapply(locs, function(z) z$country %||% NA_character_, character(1)))
      countries <- countries[!is.na(countries)]
      cities <- unique(vapply(locs, function(z) z$city %||% NA_character_, character(1)))
      cities <- cities[!is.na(cities)]

      tibble(
        n_sites   = length(locs),
        countries = if (length(countries) == 0) NA_character_ else paste(countries, collapse = "; "),
        cities    = if (length(cities) == 0) NA_character_ else paste(cities, collapse = "; ")
      )
    },

    referencesModule = function(st) {
      refs <- st$protocolSection$referencesModule$references %||% list()

      pmids <- vapply(refs, function(r) r$pmid %||% NA_character_, character(1))
      pmids <- pmids[!is.na(pmids) & nzchar(pmids)]

      citations <- vapply(refs, function(r) r$citation %||% NA_character_, character(1))
      citations <- citations[!is.na(citations) & nzchar(citations)]

      tibble(
        n_references = length(refs),
        pmids        = if (length(pmids) == 0) NA_character_ else paste(unique(pmids), collapse = "; "),
        citations    = if (length(citations) == 0) NA_character_ else paste(unique(citations), collapse = " | ")
      )
    }
  )

  supported <- names(extractors)

  if (length(modules) == 1 && identical(modules, "all")) {
    modules <- supported
  }

  bad <- setdiff(modules, supported)
  if (length(bad) > 0) {
    stop(
      "Unknown module(s): ", paste(bad, collapse = ", "),
      "\nSupported modules: ", paste(supported, collapse = ", ")
    )
  }

  out <- map_dfr(studies, function(st) {
    cols <- lapply(modules, function(m) extractors[[m]](st))
    bind_cols(cols)
  })

  # de-dup by nct_id when present
  if ("nct_id" %in% names(out)) {
    out <- distinct(out, nct_id, .keep_all = TRUE)
  }

  out
}


- testing the function

In [28]:
tbl_basic <- studies_to_table(result$studies)

tbl_with_sites <- studies_to_table(
  result$studies,
  modules = c("identificationModule", "statusModule", "contactsLocationsModule")
)

tbl_all <- studies_to_table(result$studies, modules = "all")


In [31]:
tbl_all

nct_id,org_study_id,brief_title,official_title,acronym,overall_status,status_verified_ym,start_date,start_date_type,primary_completion_date,⋯,sex,minimum_age,maximum_age,std_ages,n_sites,countries,cities,n_references,pmids,citations
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>
NCT05444894,EM-301-BThal-001,EDIT-301 for Autologous Hematopoietic Stem Cell Transplant (HSCT) in Participants With Transfusion-Dependent Beta Thalassemia (TDT),"A Multicenter Study to Evaluate the Safety, Tolerability, and Efficacy of a Single Dose of Autologous Clustered Regularly Interspaced Short Palindromic Repeats Gene-edited Cluster of Differentiation 34 (CD34+) Human Hematopoietic Stem and Progenitor Cells (HSPC) (EDIT-301) in Transfusion-Dependent Beta Thalassemia (TDT)",NA,ACTIVE_NOT_RECRUITING,2025-03,2022-04-29,ACTUAL,2025-09,⋯,ALL,18 Years,35 Years,ADULT,8,United States; Canada,Oakland; Minneapolis; New York; Cleveland; Philadelphia; Nashville; Toronto,0,NA,NA
NCT07150091,208887 Sub Study 6,"Sub-study of Belantamab Mafodotin (GSK2857916) in Combination With Nirogacestat, Lenalidomide, and Dexamethasone in Participants With RRMM","A Phase I/II, Randomized, Open-label Platform Study Utilizing a Master Protocol to Study Belantamab Mafodotin (GSK2857916) as Monotherapy and in Combination With Anti-Cancer Treatments in Participants With Relapsed/Refractory Multiple Myeloma (RRMM)-DREAMM5 - Sub-study 6 - Belantamab Mafodotin, Nirogacestat, Lenalidomide, and Dexamethasone in Combination",NA,ACTIVE_NOT_RECRUITING,2025-08,2022-07-19,ACTUAL,2025-04-17,⋯,ALL,18 Years,NA,"ADULT, OLDER_ADULT",12,United States; Brazil; Canada; France; Germany; Greece; Mexico; South Korea,Boston; Salvador; São Paulo; Halifax; Villejuif; Hamburg; Athens; Mexico City; Seoul; Ulsan,0,NA,NA
NCT05421598,DRI17509,Dose Ranging Study of Amlitelimab in Adult Participants With Moderate-to-severe Asthma,"A Randomized, Double-blind, Placebo-controlled, Parallel-group, Dose Ranging Study to Assess the Efficacy, Safety, and Tolerability of Subcutaneous Amlitelimab in Adult Participants With Moderate-to-severe Asthma",TIDE-asthma,COMPLETED,2025-10,2022-06-30,ACTUAL,2024-10-11,⋯,ALL,18 Years,75 Years,"ADULT, OLDER_ADULT",113,United States; Argentina; Brazil; Canada; Chile; Hungary; Italy; Japan; Mexico; Poland; South Africa; South Korea; Turkey (Türkiye); United Kingdom,"La Jolla; Los Angeles; San Jose; Stockton; Westminster; Boynton Beach; Cape Coral; Cutler Bay; Hialeah; Miami; Pembroke Pines; Boise; South Bend; Kansas City; Baltimore; Southfield; Henderson; Toledo; Edmond; Oklahoma City; Boerne; CABA; La Plata; Buenos Aires; Rosario; Ciudad Autonoma Buenos Aires; Vitória; Salvador; Natal; Passo Fundo; Porto Alegre; São Paulo; Sorocaba; Brampton; Ottawa; Toronto; Trois-Rivières; Québec; Talca; Santiago; Quillota; Budapest; Edelény; Gödöllö; Hajdunánás; Mosonmagyaróvár; Püspökladány; Százhalombatta; Szombathely; Cona (FE); Rome; Napoli; Verona; Narita-shi; Kamakura-shi; Yokohama; Nankoku-shi; Kumamoto; Sakai-shi; Chuo-ku; Kiyose-shi; Shinagawa-ku; Shinjuku-ku; Tachikawa-shi; Toshima-ku; Fukuoka; Hiroshima; Guadalajara; Chihuahua City; Durango, Durango; Tlalnepantla; Yucatán; Poznan; Krakow; Bialystok; Elblag; Gdansk; Tarnów; Benoni; Cape Town; Durban; George; Johannesburg; Middelburg; Daegu; Seoul; Seongnam-si, Gyeonggi-do; Istanbul; Izmir; Kayseri; Kocaeli; Mersin; Bradford",0,NA,NA
NCT05394350,1088-002,A Study of MK-1088 as Monotherapy and in Combination With Pembrolizumab in Participants With Advanced Solid Tumors (MK-1088-002),A Phase 1/Phase 2 Study to Evaluate the Safety and Tolerability of MK-1088 as Monotherapy and in Combination With Pembrolizumab in Participants With Advanced Solid Tumors,NA,TERMINATED,2024-11,2022-07-07,ACTUAL,2023-09-07,⋯,ALL,18 Years,NA,"ADULT, OLDER_ADULT",13,United States; Canada; Denmark; Israel; Switzerland,Miami; New York; San Antonio; Toronto; Québec; Copenhagen; Odense; Haifa

In [4]:

names(formals(studies_to_table))


[1] "studies" "modules"

## Jan-30

tested it out again

In [9]:

suppressWarnings(devtools::load_all())
spec <- list(
  query = "cancer",
  max_records = 10,
  phase = "Phase 2",
  country = "Canada",
  from_date = "2022-01-01",
  to_date = "2023-01-01",
  date_filter = "StartDate"
)
result <- trials_run_spec(spec)

tabs <- studies_to_tables_by_module(result$studies, modules = c("identificationModule","designModule"))

tabs$identificationModule
tabs$designModule

ℹ Loading tidytrials


nct_id,org_study_id,brief_title,official_title,acronym
<chr>,<chr>,<chr>,<chr>,<chr>
NCT05488314,CR109260,A Study of Amivantamab and Capmatinib Combination Therapy in Unresectable Metastatic Non-small Cell Lung Cancer,A Phase 1/2 Study Evaluating the Safety and Efficacy of Amivantamab and Capmatinib Combination Therapy in Unresectable Metastatic Non-small Cell Lung Cancer,METalmark
NCT05402124,CRC-CHAMP01,The Colorectal Cancer Chemoprevention Acceleration and Improvement Platform (CRC-CHAMP) Study,The Colorectal Cancer Chemoprevention Acceleration and Improvement Platform (CRC-CHAMP) Study,NA
NCT05242471,CR109178,A Study of Combination Therapy With Guselkumab and Golimumab in Participants With Moderately to Severely Active Crohn's Disease,"A Phase 2b Randomized, Double-blind, Active-and Placebo-controlled, Parallel-group, Multicenter Study to Evaluate the Efficacy and Safety of Induction and Maintenance Combination Therapy With Guselkumab and Golimumab in Participants With Moderately to Severely Active Crohn's Disease",DUET-CD
NCT04888481,CC-19-0369,68Ga-HA-DOTATATE Imaging of Suspected Somatostatin Receptor Positive Tumors,68Ga-HA-DOTATATE Imaging of Suspected Somatostatin Receptor Positive Tumors,NA
NCT05372354,CA057-003,"A Study to Evaluate Safety, Drug Levels and Effectiveness of CC-92480 (BMS-986348) in Combination With Other Treatments in Participants With Relapsed or Refractory Multiple Myeloma","An Exploratory Phase 1b/2a Multicenter, Open-Label, Novel-Novel Combination Study to Assess the Safety, Pharmacokinetics, Pharmacodynamics, and Preliminary Efficacy of CC-92480 (BMS-986348) in Novel Therapeutic Combinations in Participants With Relapsed or Refractory Multiple Myeloma",NA
NCT05419375,BX43361,Screening Study for Participants With Malignant Tumors,Master Screening Study to Determine Biomarker Status and Potential Trial Eligibility for Patients With Malignant Tumors,NA
NCT05403385,A2A-005,Study of Inupadenant (EOS100850) With Chemotherapy as Second Line Treatment for Nonsquamous Non-small Cell Lung Cancer,"A Randomized, Double-blind, Placebo-controlled, Phase 2 Study Evaluating Efficacy and Safety of Inupadenant in Combination With Carboplatin and Pemetrexed in Adults With Nonsquamous Non-small Cell Lung Cancer Who Have Progressed on Immunotherapy",NA
NCT05099003,NCI-2021-11337,A Study of the Drug Selinexor With Radiation Therapy in Patients With Newly-Diagnosed Diffuse Intrinsic Pontine (DIPG) Glioma and High-Grade Glioma (HGG),A Phase 1/2 Trial of Selinexor (KPT-330) and Radiation Therapy in Newly-Diagnosed Pediatric Diffuse Intrinsic Pontine Glioma (DIPG) and High-Grade Glioma (HGG),NA
NCT05070845,C1131003,Safety and Efficacy Study of PF-06835375 in Primary Immune Thrombocytopenia,"AN INTERVENTIONAL PHASE 2, OPEN-LABEL, MULTI-CENTER STUDY TO EVALUATE SAFETY AND EFFICACY OF PF-06835375 IN ADULT PARTICIPANTS WITH MODERATE TO SEVERE PRIMARY IMMUNE THROMBOCYTOPENIA",NA


nct_id,study_type,phases,allocation,intervention_model,primary_purpose,masking,enrollment_count,enrollment_type
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NCT05488314,INTERVENTIONAL,"PHASE1, PHASE2",NON_RANDOMIZED,SEQUENTIAL,TREATMENT,NONE,57,ACTUAL
NCT05402124,INTERVENTIONAL,PHASE2,NA,SINGLE_GROUP,HEALTH_SERVICES_RESEARCH,NONE,100,ESTIMATED
NCT05242471,INTERVENTIONAL,PHASE2,RANDOMIZED,PARALLEL,TREATMENT,DOUBLE,703,ACTUAL
NCT04888481,INTERVENTIONAL,PHASE2,NA,SINGLE_GROUP,DIAGNOSTIC,NONE,600,ESTIMATED
NCT05372354,INTERVENTIONAL,"PHASE1, PHASE2",RANDOMIZED,PARALLEL,TREATMENT,NONE,260,ESTIMATED
NCT05419375,INTERVENTIONAL,PHASE2,NA,SINGLE_GROUP,OTHER,NONE,15000,ESTIMATED
NCT05403385,INTERVENTIONAL,PHASE2,RANDOMIZED,PARALLEL,TREATMENT,QUADRUPLE,36,ACTUAL
NCT05099003,INTERVENTIONAL,"PHASE1, PHASE2",NA,SINGLE_GROUP,TREATMENT,NONE,132,ESTIMATED
NCT05070845,INTERVENTIONAL,PHASE2,NA,SINGLE_GROUP,TREATMENT,NONE,61,ESTIMATED
